In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install fastparquet

In [ ]:
!sudo apt-get install libsnappy-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsnappy-dev is already the newest version (1.1.7-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [ ]:
!python3 -m pip install  python-snappy

In [ ]:
!python3 -m pip install pyarrow

In [ ]:
!pip install --upgrade numpy

In [ ]:
#!git clone https://github.com/RobeeF/phyto_curves_reco

In [ ]:
import os
os.chdir('/content/gdrive/My Drive/phyto_curves_reco')

import re
import pandas as pd
from pred_functions import predict
from tensorflow.keras.models import load_model, model_from_json
from time import time

In [ ]:
os.chdir('/content/gdrive/My Drive/')

In [ ]:
# Model and nomenclature loading
cnn = load_model('/content/gdrive/My Drive/cnn_hyperopt_model_categ_best')

In [ ]:
tn = pd.read_csv('/content/gdrive/My Drive/data/SWINGS/L1_retreated/75 percent_6 sets/train_test_nomenclature.csv')
tn.columns = ['Particle_class', 'label']

# Define where to look the data at and where to store preds
export_folder = "/content/gdrive/My Drive/data/SWINGS/Preds/To do/L1/Missing"
export_files = os.listdir(export_folder)

pulse_regex = "_Pulse" 
files_to_pred = [file for file in export_files if re.search(pulse_regex, file) and re.search('.parq', file)] # The files containing the data to predict
#files_to_pred = ['Labelled_Pulse5_2021-02-08 12h11.parq', 'Labelled_Pulse5_2021-02-08 14h11.parq', 'Labelled_Pulse5_2021-02-08 16h11.parq', 'Labelled_Pulse5_2021-02-09 16h11.parq', 'Labelled_Pulse5_2021-02-09 18h11.parq', 'Labelled_Pulse5_2021-02-09 20h11.parq','Labelled_Pulse5_2021-02-09 22h11.parq', 'Labelled_Pulse5_2021-02-10 00h11.parq', 'Labelled_Pulse5_2021-02-10 02h11.parq', 'Labelled_Pulse5_2021-02-10 04h11.parq']
print(files_to_pred)


['Labelled_Pulse5_2021-02-22 02h11.parq', 'Labelled_Pulse5_2021-02-19 14h11.parq', 'Labelled_Pulse5_2021-03-04 00h11.parq', 'Labelled_Pulse5_2021-02-28 18h11.parq', 'Labelled_Pulse5_2021-02-11 04h11.parq', 'Labelled_Pulse5_2021-02-27 14h11.parq', 'Labelled_Pulse5_2021-02-03 00h11.parq', 'Labelled_Pulse5_2021-02-01 18h11.parq', 'Labelled_Pulse5_2021-02-20 10h11.parq', 'Labelled_Pulse5_2021-02-20 02h11.parq', 'Labelled_Pulse5_2021-01-15 20h11.parq', 'Labelled_Pulse5_2021-03-06 10h11.parq', 'Labelled_Pulse5_2021-02-10 06h11.parq', 'Labelled_Pulse5_2021-02-18 14h11.parq', 'Labelled_Pulse5_2021-01-27 22h11.parq', 'Labelled_Pulse5_2021-02-16 02h11.parq', 'Labelled_Pulse5_2021-02-03 12h11.parq', 'Labelled_Pulse5_2021-02-06 18h11.parq', 'Labelled_Pulse5_2021-03-05 18h11.parq', 'Labelled_Pulse5_2021-01-24 20h11.parq', 'Labelled_Pulse5_2021-01-14 14h11.parq', 'Labelled_Pulse5_2021-03-05 22h11.parq', 'Labelled_Pulse5_2021-01-29 22h11.parq', 'Labelled_Pulse5_2021-01-26 08h11.parq', 'Labelled_Pulse

In [ ]:
files_to_pred = files_to_pred[:(len(files_to_pred) // 2)]

In [ ]:
# Create a log file in the destination folder: list of the already predicted files
preds_store_folder = "/content/gdrive/My Drive/data/SWINGS/Preds/Done/75 percent_6 sets/Missing"  # Where to store the predictions
log_path = preds_store_folder + "/pred_logs.txt" # Register where write the already predicted files

if not(os.path.isfile(log_path)):
    open(log_path, 'w+').close()

In [ ]:
start_time = time()
nb_files_to_pred = len(files_to_pred)
print(nb_files_to_pred)

290


In [ ]:
for idx, file in enumerate(files_to_pred):
    print('Currently predicting ' + file + ' ' + str(idx))
    path = export_folder + '/' + file
    is_already_pred = False
    
    # Check if file has already been predicted
    with open(log_path, "r") as log_file:
        if file in log_file.read(): 
            is_already_pred = True
    
    if not(is_already_pred): # If not, perform the prediction
        # Predict the values
        if os.stat(path).st_size<200*10**6:
          predict(path, preds_store_folder,  cnn, tn, scale = False,\
              is_ground_truth = False)

          # Write in the logs that this file is already predicted
          with open(log_path, "a") as log_file:
              log_file.write(file + '\n')

        step_time = time()
        average_pred_time = (step_time - start_time) / (idx + 1)
        remaining_time = average_pred_time * (nb_files_to_pred - idx - 1)
        print('Average per file pred time', average_pred_time, idx, 'files already predicted')
        print('Remaining time before end of pred', remaining_time)
          
    else:
        print(file, 'already predicted')

Currently predicting Labelled_Pulse5_2021-02-22 02h11.parq 0
Labelled_Pulse5_2021-02-22 02h11.parq already predicted
Currently predicting Labelled_Pulse5_2021-02-19 14h11.parq 1
Labelled_Pulse5_2021-02-19 14h11.parq already predicted
Currently predicting Labelled_Pulse5_2021-03-04 00h11.parq 2
Labelled_Pulse5_2021-03-04 00h11.parq already predicted
Currently predicting Labelled_Pulse5_2021-02-28 18h11.parq 3
Labelled_Pulse5_2021-02-28 18h11.parq already predicted
Currently predicting Labelled_Pulse5_2021-02-11 04h11.parq 4
Labelled_Pulse5_2021-02-11 04h11.parq already predicted
Currently predicting Labelled_Pulse5_2021-02-27 14h11.parq 5
Labelled_Pulse5_2021-02-27 14h11.parq already predicted
Currently predicting Labelled_Pulse5_2021-02-03 00h11.parq 6
Labelled_Pulse5_2021-02-03 00h11.parq already predicted
Currently predicting Labelled_Pulse5_2021-02-01 18h11.parq 7
Labelled_Pulse5_2021-02-01 18h11.parq already predicted
Currently predicting Labelled_Pulse5_2021-02-20 10h11.parq 8
Lab